# Končno poročilo: Nevarnost Slovenije

## 1. Predstavitev problema

S pomočjo baze podatkov, ki opisuje pretekla kriminalna dejanja skušamo na zemljevidu prikazati čimbolj uporabne informacije o krminalu na nekem območju.
Poleg tega smo za poljubno destinacijo na slovenskem ozemlju podali oceno varnosti lokacije.

## 2. Opis podatkov

### Splošen opis
Podatke smo črpali iz strani iz https://podatki.gov.si/, bolj natančno https://podatki.gov.si/dataset/mnzpkazniva-dejanja-od-leta-2009-dalje. Uporabili smo podatke iz let 2018, 2019, 2020, 2021. Podatki iz let 2020 in 2021 v tej zbirki manjkajo, a smo jih našli na https://www.policija.si/o-slovenski-policiji/statistika/kriminaliteta.
Podatki vsebujejo kriminalna dejanja in razne podatke o njih. Mi smo uporabili stolpce: 
* UraStoritve
* DanVTednu
* OpisKD
* UpravnaEnotaStoritve
* OpisKraja

Poleg tega smo uporabili še [Polnoletni obsojenci (znani storilci) po spolu, kaznivem dejanju in glavni kazenski sankciji (tudi pogojno obsojeni), Slovenija, letno](https://podatki.gov.si/dataset/surs1360301s), [število prebivalcev po občinah](https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/2640010S.px) in [občine slovenije](https://sl.wikipedia.org/wiki/Seznam_ob%C4%8Din_v_Sloveniji).

### Obdelava podatkov
V podatkih za kazniva dejanja je nek zločin zapisan večrat. To je ocitno, saj se zaporedna številka kaznivega dejanja ponavlja večrat. Te duplikate smo odstranili zato, da so rezultati bolj točni. Pri tem smo vsekakor izgubili del podatkov, saj bi iz tega lahko pridobili tudi podatek, da se je zločin izvedel v skupini in bi ga mogoče lahko vzeli kot bolj nevarnega.

Vsakem kaznivem dejanju, ki se pojavi v podatkih, smo izračunali utež z enačbo: $$ x = ((max\_jailtime + avg\_actual\_jailtime) * dangerousness) $$
Pomen spremenljivk v enačbi:
* $max\_jailtime$: največja zaporna kazen, ki jo obsojenec lahko prejme po zakoniku. Podatek je lahko vrednost med 0 in 100, pri katerem 100 pomeni do življenska kazen.
* $avg\_actual\_jailtime$: povprečna zaporna kazen, ki so jo v preteklosti prejeli kršitelji
* $dangerousness$: subjektivnego mnenje, kako nevarno je neko dejanje za navadnega človeka. Podatek je lahko vrednost med 0 in 1.

Da je bilo določanje uteži sploh mogoče, smo morali v podatkih formatirati stolpec "OpisKD", v katerem je zapisan člen iz kazenskega zakonika tako, da je bil enak vpisu v zakoniku.  
Primer vpisa:  
* **KZ08/204-/2/S/ - TATVINA** smo spremenili v **tatvina**  

Prav tako so nekateri vpisi bili iz zastarelih zakonikov, katere smo morali najti in spremeniti tako, da so se ujemali z vpisom v zakoniku.  
Primer vpisa:  
* **KD02/103/1// - spolni napad na osebo, mlajšo od 14 let** smo spremenili v **KZ01/173/1// - spolni napad na osebo, mlajšo od petnajst let**

Uporabili smo tudi podatke o položajih regij in občin za izris na zemljevidu v aplikaciji na Androidu. Za to smo uporabili API: [*Nominatim API*](https://nominatim.org/release-docs/latest/api/Overview).

## 3. Izvedene analize

Podnaloge, ki smo jih reševali:
* Varnost posamezne regije
* Varnost posamezne občine
* Varnost posameznega prostora
* Določanje zanimivih statistik
* Vpliv dneva v tednu na nevarnost
* Vpliv ure na nevarnost

### Varnost posamezne občine:
Najprej smo podatke prečistili po zgoraj vpisanimi postopki (druga točka poročila). Nato smo zločine v vsaki občini le prešteli.
<img src="./img/nevarnost_obcine_1.png">

Že takoj na prvi pogled je razvidno, da se največ zločina zgodi v Ljubljani. Sigurno pa na podlagi teh rezultatov ne moremo odgovoriti na vprašanje, "Katera občina je najbolj varna/nevarna?". Zato smo dobljene rezultate delili z številom prebivalcev v posamezni občini. Tu so bili rezultati že boljši.
<img src="./img/nevarnost_obcine_2.png">

A vseeno je bilo sumjivo, da je v Murski Soboti tako veliko število zločinov. Zato smo izpisali zločine, ki so se zgodili v Murski Soboti.
<img src="./img/nevarnost_obcine_3.png">

V primerjavi z drugimi je največ dejanj *Kršitev temeljnih pravic delavcev*. Za ta zločin lahko rečemo, da ni zelo nevaren za navadnega človeka, zato je kriminalna dejanja potrebno utežiti. Na koncu dobimo graf:


<img src="./img/dangerousness_of_regions.png">

Na koncu smo podatke še izvozili v [JSON datoteko](https://github.com/zrezke/PR21mlfjis/blob/main/data/regije_app_data.json) za kasnejšo uporabo v aplikaciji.


### Varnost posamezne regije:
* Postopek:
  * podatke smo najprej prečistili po zgoraj vpisanem postopku
  * sestava slovarja, ki hrani uteži posameznih kaznivih dejanj
  * računanje nevarnosti posamezne regije
    * v podatkih so bile lokacije kaznivega dejanja zapisane v občinah, katere smo s pomočjo podatkov [obcine slovenije](https://github.com/zrezke/PR21mlfjis/blob/main/data/obcine_slovenije.csv) preslikali v regijo
    * najprej smo preslikali vsa kazniva dejanja z njihovimi utežmi
    * nato pa smo za vsako regijo izračunali vsoto teh uteži
    * na koncu smo vsoto delili z številom prebivalcev
    * $ nevarnost\_regije = \frac{\sum^n_{i=0}utež\_kaznivega\_denja_i} {število\_prebivalcev\_regije} $
  * izris rezultatov

<img src="./img/dangerousness_of_regions.png">

Na koncu smo podatke še izvozili v [JSON datoteko](https://github.com/zrezke/PR21mlfjis/blob/main/data/regije_app_data.json) za kasnejšo uporabo v aplikaciji.

Postopek se nahaja v [datoteki](https://github.com/zrezke/PR21mlfjis/blob/main/data_processing/varnost_posameznih_regij.ipynb).

### Varnost posamezne občine
Postopek je skoraj enak postopku predstavljenem pod prejšnjo točko **Varnost posamezne regije**, le da je korak, kjer so občine preslikane v regije izpuščen.  
Rezultati:  
<img src="./img/nevarnost_posamezne_obcine.png">

Postopek se nahaja v [datoteki](https://github.com/zrezke/PR21mlfjis/blob/main/data_processing/varnost_posameznih_obcin.ipynb).


### Vpliv dneva v tednu na nevarnost
Najprej smo le prešteli zločine. Čez teden, torej od ponedeljka do petka, je bilo zločinov več, čez vikend pa manj. 
Dobljen graf:  
<img src="./img/nevarnost_danVTednu_1.png">

Nato pa sem isto izračunal še z uteženimi vrednostmi. Rezultati so bili podobni, a je bila dejanska nevarnost čez vikend večja v primerjavi z tistimi čez teden.

<img src="./img/nevarnost_danVTednu_2.png">

To je pa zato, ker je več, manj nevarnih kriminalnih dejanj čez teden kot pa čez vikend. Ko pa dejanja utežimo, pa vrednosti pridejo bolj skupaj.

Na koncu pa smo isto izračunali še za vsako občino posebaj, kar pa bomo uporabili v aplikaciji in je za prikaz nepraktično. Zapisani rezultati so v datoteki [obc_dnevi.json](https://github.com/zrezke/PR21mlfjis/blob/main/data/obc_dnevi.json).

### Vpliv ure na nevarnost


<img src="./img/tod_nevarnostpng.png">

## 4. Rezultati